In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sklearn
import os

print(tf.__version__)

2.5.0


**本文部分内容来源于《机器学习实战》**

# 1、数据类型

## 1.1 constant张量
tensorflow中张量的使用和numpy有很多相似之处，他们之间可以互相转换，切片、操作也有很多类似的地方。

### 1.1.1 张量Tensor的创建

使用tf.constant()可以创建张量，这个张量里面的数据是constant，不可修改的，也就是不能用于模型的可训练参数，可训练参数可以使用下面介绍的tf.Variable()创建。

**指定数值创建Tensor**

可以看出，Tensor里面除了保存数据本身，还有shape和dtype属性，而浮点数默认是32位。

In [7]:
t = tf.constant([[1.,2.,3.],[4.,5.,6.]])
print(t)
t

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)


<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

**从numpy数组创建Tensor**

我们可以在numpy和Tensor间互相转换：

In [13]:
a = np.array([3, 4, 5])
t = tf.constant(a)
print(t)
t_a = t.numpy()
t_a

tf.Tensor([3 4 5], shape=(3,), dtype=int64)


array([3, 4, 5])

**通过函数创建特殊Tensor**

我们可以通过函数创建一些特殊的tensor，当然，也可以通过numpy类似的函数创建数组后再转为Tensor。

In [20]:
tf.zeros([2,3], dtype=tf.float32)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [22]:
tf.ones([2,3], dtype=tf.int64)

<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
array([[1, 1, 1],
       [1, 1, 1]])>

In [23]:
tf.random.normal([2,3])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-1.4351367 ,  0.41459087,  0.8239209 ],
       [ 0.22769402,  0.72388166,  0.68043166]], dtype=float32)>

### 1.1.2 张量的切片
张量的切片方式与numpy数值基本一致。

In [25]:
t = tf.random.normal([3,3])
print(t)

tf.Tensor(
[[ 1.1180412  -0.8491094  -1.5305184 ]
 [-0.25955534  2.1991487   0.83665794]
 [-0.2607961   1.496292   -0.83750445]], shape=(3, 3), dtype=float32)


In [26]:
t[:,2]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-1.5305184 ,  0.83665794, -0.83750445], dtype=float32)>

In [27]:
t[1,:]

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-0.25955534,  2.1991487 ,  0.83665794], dtype=float32)>

### 1.1.3 张量的运算

张量可以应该各种基本运算，比如加减乘除、平方、转置、求和、求均值等。

其中加减乘除等运行，等效于add(), multipyl()等方法。

In [34]:
a = tf.constant([[1,2,3],[4,5,6]])
b = tf.ones([2,3], dtype=tf.int32)
a + b

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[2, 3, 4],
       [5, 6, 7]], dtype=int32)>

上述的加号，python调用了a.__add__(b), 此方法仅调用了tf.add(a,b)。

In [35]:
tf.add(a,b)

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[2, 3, 4],
       [5, 6, 7]], dtype=int32)>

In [36]:
a + 5

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[ 6,  7,  8],
       [ 9, 10, 11]], dtype=int32)>

In [44]:
c = tf.constant([[1,2,3],[4,5,6]], dtype=tf.float32)
tf.exp(c)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  2.7182817,   7.389056 ,  20.085537 ],
       [ 54.59815  , 148.41316  , 403.4288   ]], dtype=float32)>

In [46]:
tf.sqrt(c)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1.       , 1.4142135, 1.7320508],
       [2.       , 2.236068 , 2.4494898]], dtype=float32)>

In [47]:
tf.pow(c,2)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [48]:
tf.square(c)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [49]:
tf.reduce_sum(c)

<tf.Tensor: shape=(), dtype=float32, numpy=21.0>

某些函数的名称与NumPy中的名称不同。例如，tf.reduce_mean（）、tf.reduce_sum（）、tf.reduce_max（）和tf.math.log（）等效于np.mean（）、np.sum（）、np.max（）和np.log（）。名称不同时，通常有充分的理由。例如，在TensorFlow中，你必须编写tf.transpose（t），不能就像在NumPy中一样只是写t.T。原因是tf.transpose（）函数与NumPy的T属性没有完全相同的功能：在TensorFlow中，使用自己的转置数据副本创建一个新的张量，而在NumPy中，t.T只是相同数据的转置视图。类似地，tf.reduce_sum（）操作之所以这样命名，是因为其GPU内核（即GPU实现）使用的reduce算法不能保证元素添加的顺序：因为32位浮点数的精度有限，因此每次你调用此操作时，结果可能会稍有不同。tf.reduce_mean（）也是如此（当然tf.reduce_max（）是确定性的）。

## 1.2 变量
变量中的数字可以被重新赋值，比如常用于模型参数。

In [26]:
# Variables
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
print(v)
print(v.value())
print(v.numpy())

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
[[1. 2. 3.]
 [4. 5. 6.]]


tf.Variable的行为与tf.Tensor的行为非常相似：你可以使用它执行相同的操作，它在NumPy中也可以很好地发挥作用，并且对类型也很挑剔。但是也可以使用assign（）方法（或assign_add（）或assign_sub（），给变量增加或减少给定值）进行修改。你还可以通过使用单元（或切片）的assign（）方法（直接指定将不起作用）或使用scatter_update（）或scatter_nd_update（）方法来修改单个单元（或切片）。

In [27]:
# assign value
v.assign(2*v)
print(v.numpy())
v[0, 1].assign(42)
print(v.numpy())
v[1].assign([7., 8., 9.])
print(v.numpy())

[[ 2.  4.  6.]
 [ 8. 10. 12.]]
[[ 2. 42.  6.]
 [ 8. 10. 12.]]
[[ 2. 42.  6.]
 [ 7.  8.  9.]]


In [28]:
try:
    v[1] = [7., 8., 9.]
except TypeError as ex:
    print(ex)

'ResourceVariable' object does not support item assignment


实际上，你几乎不需要手动创建变量，因为Keras提供了add_weight（）方法，我们将看到该方法会为你解决这个问题。而且模型参数通常由优化器直接更新，因此你几乎不需要手动更新变量。

## 1.3 张量的类型转换

类型转换会严重影响性能，并且自动完成转换很容易被忽视。为了避免这种情况，TensorFlow不会自动执行任何类型转换：如果你对不兼容类型的张量执行操作，会引发异常。例如，你不能把浮点张量和整数张量相加，甚至不能相加32位浮点和64位浮点。

如果你确定需要类型转换，可以使用f.cast()：

In [51]:
t1 = tf.constant([1., 2.], dtype=tf.float64)
t2 = tf.constant([3., 4.], dtype=tf.float32)
#t1+t2  # InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a double tensor but is a float tensor [Op:AddV2]
t1 + tf.cast(t2, tf.float64)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([4., 6.])>

In [ ]:
Tensorflow的Constant可以和python数组一样切分

In [6]:
t = tf.constant([[1.0,2.0,3.0],[4.0,5.0,6.0]])
print(t)
print(t[1:])
print(t[:1])
print(t[...,1])

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)
tf.Tensor([[4. 5. 6.]], shape=(1, 3), dtype=float32)
tf.Tensor([[1. 2. 3.]], shape=(1, 3), dtype=float32)
tf.Tensor([2. 5.], shape=(2,), dtype=float32)


常量可以很方便的与numpy的数组转换：

In [15]:
#将constant转换为numpy数组
print(t.numpy())

#将numpy转换为constant
n = np.array([[1., 2., 3.],[4., 5., 6.]])
print(tf.constant(n))

#部分numpy的操作可以直接作用于constant。其实就是先将constant转换为了numpy数组后再执行，所以结果也是一个numpy数组
print(np.square(t))


[[1. 2. 3.]
 [4. 5. 6.]]
tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float64)
[[ 1.  4.  9.]
 [16. 25. 36.]]


使用constant表示string：

In [16]:
t = tf.constant("cafe")
print(t)
print(tf.strings.length(t))
print(tf.strings.length(t, unit="UTF8_CHAR"))
print(tf.strings.unicode_decode(t, "UTF8"))

# string array
t = tf.constant(["cafe", "coffee", "咖啡"])
print(tf.strings.length(t, unit="UTF8_CHAR"))
r = tf.strings.unicode_decode(t, "UTF8")
print(r)

tf.Tensor(b'cafe', shape=(), dtype=string)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([ 99  97 102 101], shape=(4,), dtype=int32)
tf.Tensor([4 6 2], shape=(3,), dtype=int32)
<tf.RaggedTensor [[99, 97, 102, 101], [99, 111, 102, 102, 101, 101], [21654, 21857]]>


## 1.4 其它数据结构

### 1.4.1 regged
regged用于表示数组内每个维度可以有不同的长度：

In [18]:
r = tf.ragged.constant([[11, 12], [21, 22, 23], [], [41]])
# index op
print(r)
print(r[1])
print(r[1:2])

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41]]>
tf.Tensor([21 22 23], shape=(3,), dtype=int32)
<tf.RaggedTensor [[21, 22, 23]]>


In [19]:
# ops on ragged tensor
r2 = tf.ragged.constant([[51, 52], [], [71]])
print(tf.concat([r, r2], axis = 0))

<tf.RaggedTensor [[11, 12], [21, 22, 23], [], [41], [51, 52], [], [71]]>


In [20]:
r3 = tf.ragged.constant([[13, 14], [15], [], [42, 43]])
print(tf.concat([r, r3], axis = 1))

<tf.RaggedTensor [[11, 12, 13, 14], [21, 22, 23, 15], [], [41, 42, 43]]>


In [21]:
print(r.to_tensor())

tf.Tensor(
[[11 12  0]
 [21 22 23]
 [ 0  0  0]
 [41  0  0]], shape=(4, 3), dtype=int32)


### 1.4.2 sparse tensor

In [22]:
# sparse tensor
s = tf.SparseTensor(indices = [[0, 1], [1, 0], [2, 3]],
                    values = [1., 2., 3.],
                    dense_shape = [3, 4])
print(s)
print(tf.sparse.to_dense(s))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 1. 0. 0.]
 [2. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


sparse tensor的常用操作：

In [23]:
# ops on sparse tensors

s2 = s * 2.0
print(s2)

try:
    s3 = s + 1
except TypeError as ex:
    print(ex)

s4 = tf.constant([[10., 20.],
                  [30., 40.],
                  [50., 60.],
                  [70., 80.]])
print(tf.sparse.sparse_dense_matmul(s, s4))

SparseTensor(indices=tf.Tensor(
[[0 1]
 [1 0]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([2. 4. 6.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
unsupported operand type(s) for +: 'SparseTensor' and 'int'
tf.Tensor(
[[ 30.  40.]
 [ 20.  40.]
 [210. 240.]], shape=(3, 2), dtype=float32)


sparse tensor的indeice参数需要按顺序，否则就会报错。可以使用reorder()重新排序：

In [24]:
# sparse tensor
s5 = tf.SparseTensor(indices = [[0, 2], [0, 1], [2, 3]],
                    values = [1., 2., 3.],
                    dense_shape = [3, 4])
print(s5)
s6 = tf.sparse.reorder(s5)
print(tf.sparse.to_dense(s6))

SparseTensor(indices=tf.Tensor(
[[0 2]
 [0 1]
 [2 3]], shape=(3, 2), dtype=int64), values=tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32), dense_shape=tf.Tensor([3 4], shape=(2,), dtype=int64))
tf.Tensor(
[[0. 2. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 3.]], shape=(3, 4), dtype=float32)


### 1.4.3 其它
tensorflow还是有其它数据结构，包括tf.TensorArray, tf.string, tf.sets, tf.queue等等。

# 2、定制模型和训练算法

本部分我们介绍在定义和训练模型的各个步骤中，如何自定义模块。

再次提醒，95%以上的模型可以使用tf.keras提供的API完成，只有一些定制化的模型才需要用到本部分的内容。

## 2.1 自定义损失函数

我们以自定义huber损失函数为例。目前Huber算是不在Kreas官方API（但tf.keras实现了keras.loss.Huber类，我们假装不知道）。

我们先介绍一下Huber损失。当数据集有噪声，而且不方便去除时。均方误差可能会对大误差惩罚太多而导致模型不精确。平均绝对误差不会对异常值惩罚太多，但是训练可能需要一段时间才能收敛，并且训练后的模型可能不太精确。此时我们可以考虑使用Huber误差。

相比平方误差损失，Huber损失对于数据中异常值的敏感性要差一些。在值为0时，它也是可微分的。它基本上是绝对值，在误差很小时会变为平方值。误差使其平方值的大小如何取决于一个超参数δ，该参数可以调整。当δ~ 0时，Huber损失会趋向于MAE；当δ~ ∞（很大的数字），Huber损失会趋向于MSE。

$
L_\delta (y, f(x)) =  \left\{
\begin{aligned}
\frac{1}{2}(y-f(x))^2  \space \space \space \space & for |y-f(x)| <= \delta \\
\delta|y-f(x)|-\frac{1}{2}\delta^2   \space \space \space \space & otherwise
\end{aligned}
\right.
$

下面我们自定义Huber损失：

In [53]:
delta = 1
def huber_loss_fn(y_true, y_predict):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < delta
    squared_loss = tf.square(error) / 2
    linear_loss = delta * tf.abs(error) - 0.5*delta**2
    return tf.where(is_small_error, squared_loss, linear_loss)

最好返回每个实例包含一个损失的张量，而不是返回实例的平均损失。这样，Keras可以根据要求使用类别权重或样本权重。

定义了以上损失后，我们就可以在模型中使用了：

model.compile(loss=huber_loss_fn, optimizer='adam')

**此部分我们介绍了如何自定义一个损失函数，除此之外，我们还可以自定义一个损失类，详见下一小部分。**

## 2.2 自定义损失类及保存&加载包含自定义组件的模型

保存包含自定义损失函数的模型效果很好，因为Keras会保存函数的名称。每次加载时，都需要提供一个字典，将函数名称映射到实际函数。一般而言，当加载包含自定义对象的模型时，需要将名称映射到对象：


但在huber损失中，\delta是一个超参数，如果你想使用不同的阈值那怎么办。其中一个解决方案是创建一个函数，该函数创建已配置的损失函数：

In [57]:
def create_huber_fn(delta=1.0):
    def huber_loss_fn(y_true, y_predict):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < delta
        squared_loss = tf.square(error) / 2
        linear_loss = delta * tf.abs(error) - 0.5*delta**2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_loss_fn

然后，训练模型和加载模型的时候都要指定这个参数：

因为模型保存的时候，并不会保存函数的参数，所以加载模型的时候要指定这个参数。

但实际上这样并不方便，因为有可能使用模型的人并不知道训练模型时候的参数，此时，我们可以定义一个keras.losses.Loss类的子类，然后实现其get_config()函数来解决此问题：

In [63]:
class HuberLoss(keras.losses.Loss):
    def __int__(self, delta=1.0, **kwargs):
        self.delta = delta
        super().__init__(**kwargs)
    def call(self, y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < delta
        squared_loss = tf.square(error) / 2
        linear_loss = delta * tf.abs(error) - 0.5*delta**2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "delta":self.delta}

get_config（）方法返回一个字典，将每个超参数名称映射到其值。它首先调用父类的get_config（）方法，然后将新的超参数添加到此字典中（请注意，在Python3.5中添加了方便的{**x}语法）

此时，你就可以在编译模型是使用这个类的实例了：

当你保存模型时，阈值就会一起保存。。在加载模型时，只需要将类名映射到类本身即可：

当你保存模型时，Keras会调用损失实例的get_config（）方法，并将配置以JSON格式保存到HDF5文件中。加载模型时，它在HuberLoss类上调用from_config（）类方法：此方法由基类（Loss）实现，并创建该类的实例，并将**config传递给构造函数。

奥雷利安·杰龙. 机器学习实战：基于Scikit-Learn、Keras和TensorFlow：原书第2版 (OReilly精品图书系列) (Chinese Edition) (Kindle 位置 5829-5831). Kindle 版本. 

## 2.3 自定义激活函数、初始化、正则化、约束

**同样的，自定义这些组件也有函数和类2种方式，函数使用更简单，但不能保存参数。**

大多数Keras功能，例如损失、正则化、约束、初始化、度量、激活函数、层甚至完整模型，都可以以几乎相同的方式进行自定义。在大多数情况下，你只需要编写带有适当输入和输出的简单函数即可。以下是自定义激活函数（等同于keras.activations.softplus（）或tf.nn.softplus（））、自定义Glorot初始化（等同于keras.initializers.glorot_normal（））、自定义1正则化（等同于keras.regularizers.l1（0.01）），以及确保权重均为正的自定义约束（等同于keras.contraints.nonneg（）或tf.nn.relu（））的例子：

In [68]:
def my_softplus(z): # return value is just tf.nn.softplus(z)
    return tf.math.log(tf.exp(z) + 1.0)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

然后，你就可以在创建layer的时候使用这些自定义函数了：

In [66]:
layer = keras.layers.Dense(1, activation=my_softplus,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)

如果函数具有需要与模型一起保存的超参数，你需要继承适当的类，例如keras.regularizers.Regularizer，keras.constraints.Constraint，keras.initializers.Initializer或keras.layers.Layer（适用于任何层，包括激活函数）。就像我们为自定义损失所做的一样，这是一个用于1正则化的简单类，它保存了其factor超参数（这一次我们不需要调用父类构造函数或get_config（）方法，因为它们不是由父类定义的）：


In [67]:
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self, factor):
        self.factor = factor
    def __call__(self, weights):
        return tf.reduce_sum(tf.abs(self.factor * weights))
    def get_config(self):
        return {"factor": self.factor}

## 2.4 自定义指标

损失和指标在概念上不是一回事：损失（例如交叉熵）被梯度下降用来训练模型，因此它们必须是可微的（至少是在求值的地方），并且梯度在任何地方都不应为0。另外，如果人类不容易解释它们也没有问题。相反，指标（例如准确率）用于评估模型，它们必须更容易被解释，并且可以是不可微的或在各处具有0梯度。也就是说，在大多数情况下，定义一个自定义指标函数与定义一个自定义损失函数完全相同。

实际上，我们甚至可以将之前创建的Huber损失函数用作指标。

对于训练期间的每一批次，Keras都会计算该指标并跟踪自轮次开始以来的均值。大多数时候，这是你想要的，但不总是！例如，考虑一个二元分类器的精度。正如我们在第3章中看到的那样，精度是真正的数量除以正预测的数量（包括真正和假正）。假设该模型在第一批次中做出了5个正预测，其中4个是正确的，即80％的精度。然后假设该模型在第二批次中做出了3个正预测，但它们都是不正确的，即第二批次的精度为0％。如果仅计算这两个精度的均值，则可以得到40％。但是请稍等一下，这不是模型在这两个批次上的精度！实际上，在8个正预测（5+3）中，总共有4个真正（4+0），因此总体精度为50％，而不是40％。我们需要的是一个对象，该对象可以跟踪真正的数量和假正的数量，并且可以在请求时计算其比率。这正是keras.metrics.Precision类要做的事情:

In [72]:
precision = keras.metrics.Precision()
precision([0,1,1,1,0,1,0,1],[1,1,0,1,0,1,0,1])

<tf.Tensor: shape=(), dtype=float32, numpy=0.8>

In [73]:
precision([0,1,0,0,1,0,1,1],[1,0,1,1,0,0,0,0])

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

这被称为流式指标（或状态指标），因为它是逐批次更新的。在任何时候，我们都可以调用result（）方法来获取指标的当前值。我们还可以使用variables属性查看其变量（跟踪真正和假正的数量），并可以使用reset_states（）方法重置这些变量:

In [74]:
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [75]:
precision.variables

[<tf.Variable 'true_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>,
 <tf.Variable 'false_positives:0' shape=(1,) dtype=float32, numpy=array([4.], dtype=float32)>]

In [76]:
precision.reset_states()
precision.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

如果需要创建这样的流式度量，创建keras.metrics.Metric类的子类。这是一个跟踪Huber总损失的简单示例以及到目前为止看到的实例数量。当要求得到结果时，它返回比率，这就是平均Huber损失：

In [77]:
class HuberMetric(keras.metrics.Metric):
    def __init__(self, threshold=1.0, **kwargs):
        super().__init__(**kwargs) # handles base args (e.g., dtype)
        self.threshold = threshold
        #self.huber_fn = create_huber(threshold) # TODO: investigate why this fails
        self.total = self.add_weight("total", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")
    def huber_fn(self, y_true, y_pred): # workaround
        error = y_true - y_pred
        is_small_error = tf.abs(error) < self.threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = self.threshold * tf.abs(error) - self.threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    def update_state(self, y_true, y_pred, sample_weight=None):
        metric = self.huber_fn(y_true, y_pred)
        self.total.assign_add(tf.reduce_sum(metric))
        self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))
    def result(self):
        return self.total / self.count
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "threshold": self.threshold}

让我们看一下这段代码：
* 构造函数使用addwei_ght（）方法创建用于跟踪多个批次的度量状态所需的变量，在这种情况下，该变量包括所有Huber损失的总和（总计）以及到目前为止看到的实例数（计数）。如果愿意，你可以手动创建变量。Keras会跟踪任何设置为属性的tf.Variable（更一般而言，是任何“可跟踪”的对象，例如层或模型）。
* 当你使用此类的实例作为函数时（像我们对Precision对象所做的那样），将调用update_state（）方法。给定一个批次的标签和预测值（以及采样权重，但这个示例中我们忽略它们），它会更新变量。
* result（）方法计算并返回最终结果，在这种情况下为所有实例的平均Huber度量。当你使用度量作为函数时，首先调用update_state（）方法，然后调用result（）方法，并返回其输出。
* 我们还实现了get_config（）方法来确保threshold与模型一起被保存。·reset_states（）方法的默认实现将所有变量重置为0.0（但是你可以根据需要覆盖它）。

当你使用简单的函数定义指标时，Keras会自动为每个批次调用该指标，它会跟踪每个轮次的均值，就像我们手动进行的那样。因此HuberMetric类的唯一好处是保存threshold。但是，某些指标（如精度）不能简单地按批次平均：在这些情况下，除了实现流式指标之外，别无选择。


补充一个关于accuracy的说明：
* 如果模型输出是一个数值，而标签也是一个数值，此时可以直接使用metrics=['accuracy']或者metrics= [keras.metrics.Accuracy()]
* 如果模型输出是一个多分类的概率列表，标签是一个onehot后的列表，则此时使用category_accuracy或者keras.metrics.CategoricalAccuracy()
* 如果模型输出是一个多分类的概率列表，标签是一个数值，则此时使用sparse_category_accuracy或者keras.metrics.SparseCategoricalAccuracy()

## 2.5 自定义层
你可能偶尔会想要构建一个包含独特层的架构，而TensorFlow没有为其提供默认实现。在这种情况下，你将需要创建一个自定义层。或者你可能只是想构建一个重复的架构，其中包含重复多次的相同层块，因此将每个层块视为一个层会很方便。例如，如果模型是A，B，C，A，B，C，A，B，C层的序列，则你可能想定义一个包含A，B，C层的自定义层D你的模型将简化为D，D，D。让我们看看如何构建自定义层。

首先，某些层没有权重，例如keras.layers.Flatten或keras.layers.ReLU。**如果要创建不带任何权重的自定义层，最简单的选择是编写一个函数并将其包装在keras.layers.Lambda层中**。例如，以下层将对它的输入应用指数函数：


In [79]:
exponential_layer = keras.layers.Lambda(lambda x: tf.exp(x))

然后，可以像使用顺序API、函数式API或子类API等其他任何层一样使用此自定义层。你也可以将其用作激活函数（或者可以使用activation=tf.exp、activation=keras.activations.exponential或仅使用activation="exponential"）。当要预测的值具有非常不同的标度（例如0.001、10、1000）时，有时会在回归模型的输出层中使用指数层。

正如你现在可能已经猜到的，要构建自定义的有状态层（即具有权重的层），你需要创建keras.layers.Layer类的子类。例如以下类实现了Dense层的简化版本：


In [80]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)

    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape) # must be at the end

    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)

    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units,
                "activation": keras.activations.serialize(self.activation)}

让我们看一下这段代码：
 * 构造函数将所有超参数用作参数（在此示例中为units和activation），重要的是它还接受**kwargs参数。它调用父类构造函数，并将其传递给kwargs：这负责处理标准参数，例如input_shape、trainable和name。然后，它将超参数保存为属性，使用keras.activations.get（）函数将激活参数转换为适当的激活函数（它接受函数、标准字符串（如"relu"或"selu"，或None））[4]。
 * build（）方法的作用是通过为每个权重调用add_weight（）方法来创建层的变量。首次使用该层时，将调用build（）方法。在这一点上，Keras知道该层的输入形状，并将其传经元的数量，以便创建连接权重矩阵（即"Kernel"）：这对应于输入的最后维度的大小。在build（）方法的最后（并且仅在最后），你必须调用父类的build（）方法：这告诉Keras这一层被构建了（它只是设置了self.built=True）。
 * call（）方法执行所需的操作。在这种情况下，我们计算输入X与层内核的矩阵乘积，添加偏置向量，并对结果应用激活函数，从而获得层的输出。
 * compute_output_shape（）方法仅返回该层输出的形状。在这种情况下，它的形状与输入的形状相同，只是最后一个维度被替换为该层中神经元的数量。请注意，在tf.keras中，形状是tf.TensorShape类的实例，你可以使用as_list（）将其转换为Python列表。
 * get_config（）方法就像以前的自定义类中一样。请注意我们通过调用keras.activations.serialize（）保存激活函数的完整配置。

 现在，你可以像其他任何层一样使用MyDense层！

要创建一个具有多个输入（例如Concatenate）的层，call（）方法的参数应该是包含所有输入的元组，而同样，compute_output_shape（）方法的参数应该是一个包含每个输入的批处理形状的元组。要创建具有多个输出的层，call（）方法应返回输出列表，而compute_output_shape（）应返回批处理输出形状的列表（每个输出一个）。例如以下这个层需要两个输入并返回三个输出：


In [81]:
class MyMultiLayer(keras.layers.Layer):
    def call(self, X):
        X1, X2 = X
        return X1 + X2, X1 * X2

    def compute_output_shape(self, batch_input_shape):
        batch_input_shape1, batch_input_shape2 = batch_input_shape
        return [batch_input_shape1, batch_input_shape2]

现在可以像其他任何层一样使用此层，但是当然只能使用函数式和子类API，而不能使用顺序API（仅接受具有一个输入和一个输出的层）。如果你的层在训练期间和测试期间需要具有不同的行为（例如如果使用Dropout或BatchNormalization层），则必须将训练参数添加到call（）方法并使用此参数来决定要做什么。让我们创建一个在训练期间（用于正则化）添加高斯噪声但在测试期间不执行任何操作的层（Keras具有相同功能的层：keras.layers.GaussianNoise）：

In [83]:
class AddGaussianNoise(keras.layers.Layer):
    def __init__(self, stddev, **kwargs):
        super().__init__(**kwargs)
        self.stddev = stddev

    def call(self, X, training=None):
        if training:
            noise = tf.random.normal(tf.shape(X), stddev=self.stddev)
            return X + noise
        else:
            return X

    def compute_output_shape(self, batch_input_shape):
        return batch_input_shape

## 2.6 自定义模型

自定义模型需要继承keras.Model类，在构造函数中创建层和变量，并实现call（）方法来执行你希望模型执行的任何操作。
![](https://lujinhong-markdown.oss-cn-beijing.aliyuncs.com/md/截屏2021-08-0911.00.25.png)

输入经过第一个密集层，然后经过由两个密集层组成的残差块并执行加法运算（残差块将其输入加到其输出中），然后经过相同的残差块3次或者更多次，然后通过第二个残差块，最终结果通过密集输出层。注意，该模型没有多大意义，这只是一个示例，它说明了你可以轻松构建所需的任何模型，即使是包含循环和跳过连接的模型。要实现此模型，最好首先创建一个ResidualBlock层，因为我们将创建几个相同的块（并且我们可能想在另一个模型中重用它）：

In [84]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation="elu",
                                          kernel_initializer="he_normal")
                       for _ in range(n_layers)]

    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs + Z

该层有点特殊，因为它包含其他层。这由Keras透明地处理：它会自动检测到隐藏的属性，该属性包含可跟踪的对象（在这个示例中是层），因此它们的变量会自动添加到该层的变量列表中。这个类的其余部分不言自明。接下来让我们使用子类API定义模型本身：

In [86]:
class ResidualRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden1 = keras.layers.Dense(30, activation="elu",
                                          kernel_initializer="he_normal")
        self.block1 = ResidualBlock(2, 30)
        self.block2 = ResidualBlock(2, 30)
        self.out = keras.layers.Dense(output_dim)

    def call(self, inputs):
        Z = self.hidden1(inputs)
        for _ in range(1 + 3):
            Z = self.block1(Z)
        Z = self.block2(Z)
        return self.out(Z)

我们在构造函数中创建层，并在call（）方法中使用它们。然后就可以像使用任何其他模型一样使用此模型（对其进行编译、拟合、评估和预测）。如果你还希望能够使用save（）方法保存模型并使用keras.models.load_model（）函数加载模型，则必须在两个ResidualBlock类和ResidualRegressor类中都实现get_config（）方法（就像我们之前所做的那样）。另外，你可以使用save_weights（）和load_weights（）方法保存和加载权重。


**Model类是Layer类的子类，因此可以像定义层一样定义和使用模型。**但是模型具有一些额外的功能，包括其compile（）、fit（）、evaluate（）和predict（）方法（以及一些变体）以及get_layers（）方法（可以按名称或按索引返回任何模型的层）和save（）方法（支持keras.models.load_model（）和keras.models.clone_model（））。如果模型提供的功能比层更多，为什么不将每个层都定义为模型？从技术上讲可以，但是通常可以轻松地将模型的内部组件（即层或可重复使用的层块）与模型本身（即要训练的对象）区分开来。前者应继承Layer类，而后者应继承Model类。



## 2.7 基于模型内部的损失和指标

我们之前定义的自定义损失和指标均基于标签和预测（以及可选的样本权重）。有时候，你可能要根据模型的其他部分来定义损失，例如权重或隐藏层的激活。这对于进行正则化或监视模型的某些内部方面可能很有用。

要基于模型内部定义自定义损失，根据所需模型的任何部分进行计算，然后将结果传递给add_loss（）方法。例如，让我们构建一个自定义回归MLP模型，该模型由5个隐藏层和一个输出层的堆栈组成。此自定义模型还将在上部隐藏层的顶部有辅助输出。与该辅助输出相关的损失称为重建损失：它是重建与输入之间的均方差。通过将这种重建损失添加到主要损失中，我们鼓励模型通过隐藏层保留尽可能多的信息，即使对回归任务本身没有直接用处的信息。实际中，这种损失有时会提高泛化性（这是正则化损失）。以下是带有自定义重建损失的自定义模型的代码：

In [87]:
class ReconstructingRegressor(keras.models.Model):
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                                          kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        # TODO: check https://github.com/tensorflow/tensorflow/issues/26260
        #self.reconstruction_mean = keras.metrics.Mean(name="reconstruction_error")

    def build(self, batch_input_shape):
        n_inputs = batch_input_shape[-1]
        self.reconstruct = keras.layers.Dense(n_inputs)
        super().build(batch_input_shape)

    def call(self, inputs, training=None):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        reconstruction = self.reconstruct(Z)
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))
        self.add_loss(0.05 * recon_loss)
        #if training:
        #    result = self.reconstruction_mean(recon_loss)
        #    self.add_metric(result)
        return self.out(Z)

让我们看一下这段代码：
 * 构造函数创建具有5个密集隐藏层和一个密集输出层的DNN。
 * build（）方法创建一个额外的密集层，该层用于重建模型的输入。必须在此处创建它，因为它的单元数必须等于输入数，并且在调用build（）方法之前，此数是未知的。
 * call（）方法处理所有5个隐藏层的输入，然后将结果传递到重建层，从而产生重构。
 * 然后call（）方法计算重建损失（重建与输入之间的均方差），并使用add_loss（）方法[7]将其添加到模型的损失列表中。请注意，我们通过将其乘以0.05（这是你可以调整的超参数）按比例缩小了重建。这确保了重建损失不会在主要损失中占大部分。
 * 最后，call（）方法将隐藏层的输出传递到输出层并返回其输出。

 同样，你可以通过以所需的任何方式计算来添加基于模型内部的自定义指标，只要结果是指标对象的输出即可。例如，你可以在构造函数中创建keras.metrics.Mean对象，然后在call（）方法中调用它，将recon_loss传递给它，最后通过调用模型的add_metric（）方法将其添加到模型中。这样当你训练模型时，Keras会同时显示每个轮次的平均损失（损失是主要损失加上0.05倍的重建损失）和每个轮次的平均重建误差。
 
 在超过99％的情况下，到目前为止，我们讨论的所有内容都足以实现你要构建的任何模型，即使具有复杂的架构、损失和指标。但是在极少数情况下，你可能需要自定义训练循环，在那之前，我们需要研究如何在TensorFlow中自动计算梯度。

## 2.8 使用自动微分计算梯度

为了示范如何使用自动微分计算梯度，我们看一下这个简单的函数：

In [89]:
def f(w1, w2):
    return 3*w1**2+2*w1*w2

如果你了解微积分，则可以通过分析发现该函数关于w1的偏导数为6*w1+2*w2。你还可以发现其关于w2的偏导数是2*w1。例如，在点（w1，w2）=（5，3）处，这些偏导数分别等于36和10，因此此点的梯度向量为（36，10）。但是如果这是一个神经网络，则该函数将更加复杂，通常具有数以万计的参数，并且用手工分析找到偏导数将几乎是不可能的任务。一种解决方案是通过在调整相应参数时测量函数输出的变化来计算每个偏导数的近似值：

In [91]:
w1, w2 = 5, 3
eps = 1e-6
(f(w1+eps, w2)-f(w1,w2))/eps

36.000003007075065

In [92]:
(f(w1, w2+eps)-f(w1,w2))/eps

10.000000003174137

看起来不错！这工作得很好并且易于实现，但这只是一个近似值，重要的是每个参数至少要调用一次f（）（不是两次，因为我们只计算一次f（w1，w2））。每个参数至少需要调用f（）一次，这种方法对于大型神经网络来说很棘手。因此，我们应该使用自动微分。TensorFlow使这个变得非常简单：


In [94]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
    z = f(w1, w2)
    
gradient = tape.gradient(z, [w1, w2])
gradient

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

我们首先定义两个变量w1和w2，然后创建一个tf.GradientTape上下文，该上下文将自动记录涉及变量的每个操作，最后我们要求该tape针对两个变量[w1，w2]计算结果z的梯度。

结果不仅是准确的（精度仅受浮点误差限制），而且无论有多少变量，gradient（）方法都只经历一次已经记录的计算（以相反的顺序），因此它非常有效。就像魔术一样！

调用tape的gradient（）方法后，tape会立即被自动擦除，因此如果你尝试两次调用gradient（），则会出现异常。

如果你需要多次调用gradient（），则必须使该tape具有持久性，并在每次使用完该tape后将其删除以释放资源。

In [95]:
with tf.GradientTape(persistent=True) as tape:
    z = f(w1, w2)

dz_dw1 = tape.gradient(z, w1)
dz_dw2 = tape.gradient(z, w2) # works now!
del tape

默认情况下，tape仅跟踪涉及变量的操作，因此如果你尝试针对变量以外的任何其他变量计算z的梯度，则结果将为None。

类如下例涉及常量的计算：

In [96]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
    z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])
print(gradients)

[None, None]


但是，你可以强制tape观察你喜欢的任何张量，来记录涉及它们的所有操作。然后你可以针对这些张量计算梯度，就好像它们是变量一样：

In [98]:
with tf.GradientTape() as tape:
    tape.watch(c1)
    tape.watch(c2)
    z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])
print(gradients)

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>, <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]


在某些情况下，这可能很有用，如果你要实现正则化损失，从而在输入变化不大时惩罚那些变化很大的激活：损失将基于激活相对于输入的梯度而定。由于输入不是变量，因此你需要告诉tape观察它们。

大多数情况下，一个梯度tape是用来计算单个值（通常是损失）相对于一组值（通常是模型参数）的梯度。这就是反向模式自动微分有用的地方，因为它只需执行一次正向传播和一次反向传播即可一次获得所有梯度。如果你尝试计算向量的梯度，例如包含多个损失的向量，那么TensorFlow将计算向量和的梯度。因此如果你需要获取单独的梯度（例如每种损失相对于模型参数的梯度），则必须调用tape的jacobian（）方法：它对向量中的每个损失执行一次反向模式自动微分（默认情况下全部并行）。它甚至可以计算二阶偏导数（Hessian，即偏导数的偏导数），但实际上很少需要（请参阅notebook的“ComputingGradientswithAutodiff”部分）。

在某些情况下，你可能希望阻止梯度在神经网络的某些部分反向传播。为此必须使用tf.stop_gradient（）函数。该函数在前向传递过程中返回其输入（如tf.identity（）），但在反向传播期间不让梯度通过（它的作用类似于常量）

In [100]:
def f(w1, w2):
    return 3 * w1 ** 2 + tf.stop_gradient(2 * w1 * w2)

with tf.GradientTape() as tape:
    z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
print(gradients)

[<tf.Tensor: shape=(), dtype=float32, numpy=30.0>, None]


最后在计算梯度时，你有时可能会遇到一些数值问题。例如，如果你用大数值输入来计算my_softplus（）函数的梯度，则结果为NaN：

In [101]:
x = tf.Variable(100.)
with tf.GradientTape() as tape:
    z = my_softplus(x)

tape.gradient(z, [x])

[<tf.Tensor: shape=(), dtype=float32, numpy=nan>]

这是因为使用自动微分计算此函数的梯度会导致一些数值上的困难：由于浮点精度误差，自动微分最终导致计算无穷除以无穷（返回NaN）。幸运的是，我们可以分析发现softplus函数的导数为1/（1+1/exp（x）），在数值上是稳定的。接下来我们可以告诉TensorFlow在计算my_softplus（）函数的梯度时使用@tf.custom_gradient来修饰它并使它既返回其正常输出又返回计算导数的函数（注意，它将接收到目前为止反向传播的梯度，直到softplus函数。根据链式规则，我们应该将它们乘以该函数的梯度）：


In [102]:
@tf.custom_gradient
def my_better_softplus(z):
    exp = tf.exp(z)
    def my_softplus_gradients(grad):
        return grad / (1 + 1 / exp)
    return tf.math.log(exp + 1), my_softplus_gradients

现在当我们计算my_better_softplus（）函数的梯度时，即使对于较大的输入值，我们也可以获得正确的结果（但是由于指数的关系，主要输出仍然会爆炸。一种解决方法是使用tf.where（）在输入较大时返回输入）。

恭喜你！现在你可以计算任何函数的梯度（前提是你想计算的那个点是可微的），甚至在需要时阻止反向传播，并编写自己的梯度函数！这可能比你需要的更有灵活性，即使你构建自己的自定义训练循环，正如我们下面要看到的。


## 2.9 自定义训练循环

在极少数情况下，fit（）方法可能不够灵活而无法满足你的需要。例如我们在Wide＆Deep论文使用了两种不同的优化器：一种用于宽路径，另一种用于深路径。由于fit（）方法只使用一个优化器（编译模型时指定的优化器），因此实现该论文需要编写你自己的自定义循环。

有时候你可能还需要自定义训练循环，因为可以监控循环过程中的每一个参数和步骤，并将你所需要的所有信息打印出来。

但再次提醒，除非你明确需要更大的灵活性，而且当前API无法满足你的需求，否则你还是应该使用fit()函数。

首先，我们简历一个全连接网络，但无需compile，因为我们将手动处理训练循环：


In [103]:
l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

接下来，让我们创建一个小函数，从训练集中随机采样一批实例（DataAPI提供了更好的选择）：

In [104]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

让我们定义一个函数，显示训练状态，包括步数、步总数、从轮次开始以来的平均损失（即我们将使用Mean指标来计算），和其他指标：

In [105]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics,
          end=end)

除非你不熟悉Python字符串的格式设置，否则这段代码是不言自明的：{：.4f}会格式化小数点后四位数字的浮点数，并使用\r（回车）和end=""确保状态栏始终打印在同一行上。在notebook中，print_status_bar（）函数包括一个进度条，但是你也可以使用方便的tqdm库。

首先我们需要定义一些超参数，然后选择优化器、损失函数和指标（在此示例中是MAE）：


下面我们开始创建自定义循环：

In [2]:
%
for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()
%

UsageError: Line magic function `%` not found.


这段代码中做了很多事情，所以让我们来看一下：
 * 我们创建了两个嵌套循环：一个用于轮次，另一个用于轮次内的批处理。
 * 然后从训练集中抽取一个随机批次。
 * 在tf.GradientTape（）块中，我们对一个批次进行了预测（使用模型作为函数），并计算了损失：它等于主损失加其他损失（在此模型中，每层都有一个正则化损失）。由于mean_squared_error（）函数每个实例返回一个损失，因此我们使用tf.reduce_mean（）计算批次中的均值（如果要对每个实例应用不同的权重，则可以在这里进行操作）。正则化损失已经归约到单个标量，因此我们只需要对它们进行求和（使用tf.add_n（）即可对具有相同形状和数据类型的多个张量求和）。
 * 接下来，我们要求tape针对每个可训练变量（不是所有变量！）计算损失的梯度，然后用优化器来执行“梯度下降”步骤。
 * 然后，我们更新平均损失和指标（在当前轮次内），并显示状态栏。
 * 在每个轮次结束时，我们再次显示状态栏以使其看起来完整并打印换行符，然后重置平均损失和指标的状态。

 如果你设置优化器的超参数clipnorm或clipvalue，它会为你解决这一问题。如果要对梯度应用任何其他变换，只需在调用apply_gradients（）方法之前进行即可。如果你要对模型添加权重约束（例如在创建层时设置kernel_constraint或bias_constraint），则应更新训练循环以在apply_gradients（）之后应用这些约束：



最重要的是，此训练循环不会处理在训练期间和测试期间行为不同的层（例如BatchNormalization或Dropout）。要处理这些问题，你需要使用training=True调用模型，确保将其传播到需要它的每个层。

如你所见，有很多事情需要自己做，这很容易出错。但好处你可以完全控制，这也是你自己的要求。

既然你知道如何自定义模型的任何部分和训练算法，那么我们来看看如何使用TensorFlow的自动图生成功能：它可以大大加快自定义代码的速度，并且还可以移植到TensorFlow支持的任何平台上。



In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

l2_reg = keras.regularizers.l2(0.05)
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="elu", kernel_initializer="he_normal",
                       kernel_regularizer=l2_reg),
    keras.layers.Dense(1, kernel_regularizer=l2_reg)
])

def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]

def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics,
          end=end)
    

import time

mean_loss = keras.metrics.Mean(name="loss")
mean_square = keras.metrics.Mean(name="mean_square")
for i in range(1, 50 + 1):
    loss = 1 / i
    mean_loss(loss)
    mean_square(i ** 2)
    print_status_bar(i, 50, mean_loss, [mean_square])
    time.sleep(0.05)
    

def progress_bar(iteration, total, size=30):
    running = iteration < total
    c = ">" if running else "="
    p = (size - 1) * iteration // total
    fmt = "{{:-{}d}}/{{}} [{{}}]".format(len(str(total)))
    params = [iteration, total, "=" * p + c + "." * (size - p - 1)]
    return fmt.format(*params)

def print_status_bar(iteration, total, loss, metrics=None, size=30):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{} - {}".format(progress_bar(iteration, total), metrics), end=end)
    
    
mean_loss = keras.metrics.Mean(name="loss")
mean_square = keras.metrics.Mean(name="mean_square")
for i in range(1, 50 + 1):
    loss = 1 / i
    mean_loss(loss)
    mean_square(i ** 2)
    print_status_bar(i, 50, mean_loss, [mean_square])
    time.sleep(0.05)
    
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.MeanAbsoluteError()]


for epoch in range(1, n_epochs + 1):
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        for variable in model.variables:
            if variable.constraint is not None:
                variable.assign(variable.constraint(variable))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    for metric in [mean_loss] + metrics:
        metric.reset_states()

50/50 - loss: 0.0900 - mean_square: 858.5000
50/50 [==============================] - loss: 0.0900 - mean_square: 858.5000
Epoch 1/5
11610/11610 [==============================] - mean: 1.3955 - mean_absolute_error: 0.5722
Epoch 2/5
11610/11610 [==============================] - mean: 0.6774 - mean_absolute_error: 0.5280
Epoch 3/5
11610/11610 [==============================] - mean: 0.6351 - mean_absolute_error: 0.5177
Epoch 4/5
11610/11610 [==============================] - mean: 0.6384 - mean_absolute_error: 0.5181
Epoch 5/5
11610/11610 [==============================] - mean: 0.6440 - mean_absolute_error: 0.5222


# 3、Tensorflow的函数和图

使用tf_function与auto_graph，可以将普通python代码转成tensorflow代码，提高运行效率。

方式一：使用转化函数

In [109]:
# tf.function and auto-graph.
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z >= 0 ? scale * z : scale * alpha * tf.nn.elu(z)
    is_positive = tf.greater_equal(z, 0.0)
    return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))

print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-3., -2.5])))

scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))
print(scaled_elu_tf(tf.constant([-3., -2.5])))

print(scaled_elu_tf.python_function is scaled_elu)

%timeit scaled_elu(tf.random.normal((1000, 1000)))
%timeit scaled_elu_tf(tf.random.normal((1000, 1000)))

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
True
2.56 ms ± 144 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.95 ms ± 19.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


方式二：使用声明

In [110]:
@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total

print(converge_to_2(20))

tf.Tensor(1.9999981, shape=(), dtype=float32)


在TensorFlow1中，图是不可避免的（随之而来的是复杂性），因为它们是TensorFlowAPI的核心部分。在TensorFlow2中，它们仍在那儿，但是不那么重要了，而且使用起来要简单得多。为了说明其简单性，让我们从计算其输入的立方的简单函数开始：

In [111]:
def cube(x):
    return x ** 3

我们显然可以使用Python值（例如一个整数或浮点数）来调用此函数，也可以使用张量来调用它：

In [112]:
cube(2)

8

In [113]:
cube(tf.constant(2.0))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

现在，让我们使用tf.function（）将此Python函数转换为TensorFlow函数：

In [115]:
tf_cube = tf.function(cube)
tf_cube

In [116]:
tf_cube(2)

<tf.Tensor: shape=(), dtype=int32, numpy=8>

In [117]:
tf_cube(tf.constant(2.))

<tf.Tensor: shape=(), dtype=float32, numpy=8.0>

在后台，tf.function（）分析了cube（）函数执行的计算，并生成等效的计算图！如你所见，它相当容易（我们将很快看到它的工作原理）。另外我们可以使用tf.function作为修饰器，这在实际中更常见：

In [119]:
@tf.function
def cube(x):
    return x ** 3
cube

如果需要，可以通过TF函数的python_function属性使用原Python函数：

In [120]:
tf_cube.python_function(2.0)

8.0

TensorFlow可以优化计算图，修剪未使用的节点，简化表达式（例如1+2将替换为3），等等。准备好优化的图后，TF函数会以适当的顺序（并在可能时并行执行）有效地执行图中的操作。因此TF函数通常比原始的Python函数运行得更快，尤其是在执行复杂计算[1]的情况下。大多数时候，你并不需要真正了解很多：当你想增强Python函数时，只需将其转换为TF函数即可。

此外，当你编写自定义损失函数、自定义指标、自定义层或任何其他自定义函数，并在Keras模型中使用它时（如本章所述），Keras会自动将你的函数转换为TF函数——不需要使用tf.function（）。因此大多数时候，所有这些处理都是100％透明的。

默认情况下，TF函数会为每个不同的输入形状和数据类型集生成一个新图形，并将其缓存以供后续调用。例如，如果调用tf_cube（tf.constant（10）），将为形状为[]的int32张量生成图形。如果调用tf_cube（tf.constant（20）），则会重用相同的图。但是如果你随后调用tf_cube（tf.constant（[10，20]）），则会为形状为[2]的int32张量生成一个新图。这就是TF函数处理多态（即变化的参数类型和形状）的方式。但是这仅适用于张量参数：如果将Python数值传递给TF函数，则将为每个不同的值生成一个新图：例如，调用tf_cube（10）和tf_cube（20）将生成两个图。



** 关于自动图的内部实现可以参考《计算学习实战》chpt12.**

在大多数情况下，将执行TensorFlow操作的Python函数转换为TF函数很简单：用@tf.function修饰它或让Keras替你处理。但是有一些规则需要遵守：
 * 如果调用任何外部库，包括NumPy甚至标准库，此调用将仅在跟踪过程中运行。它不会成为图表的一部分。实际上，TensorFlow图只能包含TensorFlow构造（张量、运算、变量、数据集等）。因此请使用tf.reduce_sum（）代替np.sum（），使用tf.sort（）代替内置的sorted（）函数，以此类推（除非你真的希望代码仅在跟踪过程中运行）。这还有一些其他含义：
 * 如果你定义了一个返回np.random.rand（）的TF函数f（x），则仅在跟踪该函数时才会生成随机数，因此f（tf.constant（2.））和f（tf.constant（3.））将返回相同的随机数，但f（tf.constant（[2.，3.]））将返回不同的随机数。如果把np.random.rand（）替换为tf.random.uniform（[]），则每次操作都会生成一个新的随机数，因为该操作将成为图形的一部分。
 * 如果你的非TensorFlow代码具有副作用（例如记录某些内容或更新Python计数器），那么你不应期望每次调用TF函数时都会发生这些副作用，因为它们只会在跟踪该函数时发生。
 * 你可以在tf.py_function（）操作中包装任何Python代码，但这样做会降低性能，因为TensorFlow无法对此代码进行任何图优化。这也会降低可移植性，因为该图仅可在安装了Python（并且安装了正确的库）的平台上运行。
 * 你可以调用其他Python函数或TF函数，但它们应遵循相同的规则，因为TensorFlow会在计算图中捕获它们的操作。请注意这些其他函数不需要用@tf.function修饰。
 * 如果该函数创建了一个TensorFlow变量（或任何其他有状态的TensorFlow对象，例如数据集或队列），则必须在第一次调用时这样做（只有这样做），否则你会得到一个异常。通常最好在TF函数（例如在自定义层的build（）方法中）外部创建变量。如果要为变量分配一个新值，确保调用它的assign（）方法，而不要使用=运算符。
 * 你的Python函数的源代码应可用于TensorFlow。如果源代码不可用（例如，如果你在Pythonshell中定义函数，而该函数不提供对源代码的访问权，或者仅将已编译的*.pycPython文件部署到生产环境中），则生成图的过程会失败或功能受限。
 * TensorFlow只能捕获在张量或数据集上迭代的for循环。因此请确保你使用foriintf.range（x），而不是使用foriinrange（x），否则这个循环不会在图中被捕获。相反它会在跟踪过程中运行（也许这就是你想要的，如果要使用for循环来构建图形，例如在神经网络中创建每一层）。
 * 与通常一样，出于性能原因，应尽可能使用向量化实现，而不是使用循环。

